In [ ]:
# IMPORT LIBRARIES

In [ ]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

In [ ]:
# LOAD DATA

In [ ]:
heart_data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
heart_data.head()